In [29]:
import pandas as pd
import numpy as np
from nba_api.stats import endpoints

In [30]:
# list of seasons to pull data from API
seasons = ["2005-06", "2006-07", "2007-08", "2008-09", "2009-10", "2010-11", "2011-12", "2012-13", 
           "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22"]

# initialize df with latest full season 21/22
df = endpoints.leaguestandings.LeagueStandings(season="2004-05").standings.get_data_frame()[['SeasonID', 'TeamID', 'TeamCity', 'TeamName', 'Conference', 'PlayoffRank', 'WinPCT', 'DiffPointsPG']]

# concatenate all seasons in list to df
for i in seasons:
    temp_df = endpoints.leaguestandings.LeagueStandings(season=i).standings.get_data_frame()[['SeasonID', 'TeamID', 'TeamCity', 'TeamName', 'Conference', 'PlayoffRank', 'WinPCT', 'DiffPointsPG']]
    df = pd.concat([df, temp_df])

In [31]:
# change seasonid type to numeric
df["SeasonID"] = df["SeasonID"].str[1:]
df["SeasonID"] = pd.to_numeric(df["SeasonID"])

In [32]:
# # create teamID to teamName mapping
# idNameMapping = df[["TeamID", 'TeamName', 'Conference']][df['SeasonID'] == 2021].set_index('TeamID')
# idNameMapping.to_csv('id_name_mapping.csv')

In [33]:
# playoffrank lag lists that will become columns
PlayoffRankLag1 = []
PlayoffRankLag2 = []
PlayoffRankLag3 = []
PlayoffRankLag4 = []

# winpct lag lists that will become columns
WinPCTLag1 = []
WinPCTLag2 = []
WinPCTLag3 = []
WinPCTLag4 = []

# diffpointspg lag lists that will become columns
DiffPointsPGLag1 = []
DiffPointsPGLag2 = []
DiffPointsPGLag3 = []
DiffPointsPGLag4 = []

# 2004/05 was the first season NBA had 30 teams
# to get 3 lagged years for each team, final season has to be 2007/08

# loop through df and append 3 lags into lists
for i, row in df.iterrows():
    currentTeamID = row.TeamID
    currentSeasonID = row.SeasonID
    if currentSeasonID >= 2008:
        for n in range(1, 5):
            eval(f'PlayoffRankLag{n}').append(df[(df["SeasonID"] == currentSeasonID - n) & (df["TeamID"] == currentTeamID)].iloc[0]["PlayoffRank"]) # append lagged playoff rank into list
            eval(f'WinPCTLag{n}').append(df[(df["SeasonID"] == currentSeasonID - n) & (df["TeamID"] == currentTeamID)].iloc[0]["WinPCT"]) # append lagged winpct into list
            eval(f'DiffPointsPGLag{n}').append(df[(df["SeasonID"] == currentSeasonID - n) & (df["TeamID"] == currentTeamID)].iloc[0]["DiffPointsPG"]) # append lagged diffpointspg into list
    else:
        for n in range(1, 5):
            eval(f'PlayoffRankLag{n}').append(None)
            eval(f'WinPCTLag{n}').append(None)
            eval(f'DiffPointsPGLag{n}').append(None)

# convert lagged lists into df columns
for n in range(1, 5):
    df[f"PlayoffRankLag{n}"] = eval(f'PlayoffRankLag{n}')
    df[f"WinPCTLag{n}"] = eval(f'WinPCTLag{n}')
    df[f"DiffPointsPGLag{n}"] = eval(f'DiffPointsPGLag{n}')

# remove rows with nan values
df_nonulls = df[df['PlayoffRankLag1'].notna()]

# create target column from playoff rank
df_nonulls['isPlayoff'] = (df_nonulls['PlayoffRank'] <= 8).astype(int)

# split df into east and west conferences
df_east = df_nonulls[df_nonulls["Conference"] == "East"]
df_west = df_nonulls[df_nonulls["Conference"] == "West"]

# drop current playoff rank, winpct, diffpointspg, and conference columns
df_east = df_east.drop(columns=['PlayoffRank', 'Conference', 'WinPCT', 'DiffPointsPG', 'TeamCity', 'TeamName']).reset_index(drop=True)
df_west = df_west.drop(columns=['PlayoffRank', 'Conference', 'WinPCT', 'DiffPointsPG', 'TeamCity', 'TeamName']).reset_index(drop=True)
df_merged = df_nonulls.drop(columns=['PlayoffRank', 'Conference', 'WinPCT', 'DiffPointsPG', 'TeamCity', 'TeamName']).reset_index(drop=True)

C:\Users\mltns\AppData\Local\Temp\ipykernel_9336\3830010814.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonulls['isPlayoff'] = (df_nonulls['PlayoffRank'] <= 8).astype(int)


In [34]:
df_merged

,SeasonID,TeamID,PlayoffRankLag1,WinPCTLag1,DiffPointsPGLag1,PlayoffRankLag2,WinPCTLag2,DiffPointsPGLag2,PlayoffRankLag3,WinPCTLag3,DiffPointsPGLag3,PlayoffRankLag4,WinPCTLag4,DiffPointsPGLag4,isPlayoff
0,2008,1610612739,4.0,0.549,-0.4,2.0,0.610,3.8,3.0,0.610,2.2,9.0,0.512,0.8,1
1,2008,1610612747,1.0,0.695,7.3,7.0,0.512,-0.1,7.0,0.549,2.5,11.0,0.415,-3.0,1
2,2008,1610612743,8.0,0.610,3.7,6.0,0.549,1.6,4.0,0.537,0.2,7.0,0.598,2.0,1
3,2008,1610612738,1.0,0.805,10.3,15.0,0.293,-3.4,11.0,0.402,-1.5,4.0,0.549,0.9,1
4,2008,1610612759,3.0,0.683,4.8,3.0,0.707,8.4,1.0,0.768,6.8,2.0,0.720,7.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2021,1610612757,6.0,0.583,1.8,8.0,0.473,-1.1,3.0,0.646,4.2,3.0,0.598,2.6,0
416,2021,1610612765,15.0,0.278,-4.5,13.0,0.303,-3.6,8.0,0.500,-0.2,9.0,0.476,-0.1,0
417,2021,1610612760,14.0,0.306,-10.6,5.0,0.611,2.0,6.0,0.598,3.4,4.0,0.585,3.4,0
418,2021,1610612745,15.0,0.236,-7.9,4.0,0.611,3.0,4.0,0.646,4.8,1.0,0.793,8.5,0


In [35]:
df_east

,SeasonID,TeamID,PlayoffRankLag1,WinPCTLag1,DiffPointsPGLag1,PlayoffRankLag2,WinPCTLag2,DiffPointsPGLag2,PlayoffRankLag3,WinPCTLag3,DiffPointsPGLag3,PlayoffRankLag4,WinPCTLag4,DiffPointsPGLag4,isPlayoff
0,2008,1610612739,4.0,0.549,-0.4,2.0,0.610,3.8,3.0,0.610,2.2,9.0,0.512,0.8,1
1,2008,1610612738,1.0,0.805,10.3,15.0,0.293,-3.4,11.0,0.402,-1.5,4.0,0.549,0.9,1
2,2008,1610612753,3.0,0.634,5.5,8.0,0.488,0.8,10.0,0.439,-1.1,10.0,0.439,-2.2,1
3,2008,1610612737,8.0,0.451,-1.8,13.0,0.366,-4.8,14.0,0.317,-4.8,15.0,0.159,-9.7,1
4,2008,1610612748,15.0,0.183,-8.6,4.0,0.537,-0.9,2.0,0.634,3.9,1.0,0.720,6.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2021,1610612752,4.0,0.569,2.3,12.0,0.318,-6.5,15.0,0.207,-9.2,11.0,0.354,-3.6,0
206,2021,1610612764,8.0,0.472,-1.8,9.0,0.347,-4.7,11.0,0.390,-2.9,8.0,0.524,0.6,0
207,2021,1610612754,9.0,0.472,0.0,4.0,0.616,2.0,5.0,0.585,3.3,5.0,0.585,1.4,0
208,2021,1610612765,15.0,0.278,-4.5,13.0,0.303,-3.6,8.0,0.500,-0.2,9.0,0.476,-0.1,0


In [36]:
df_west

,SeasonID,TeamID,PlayoffRankLag1,WinPCTLag1,DiffPointsPGLag1,PlayoffRankLag2,WinPCTLag2,DiffPointsPGLag2,PlayoffRankLag3,WinPCTLag3,DiffPointsPGLag3,PlayoffRankLag4,WinPCTLag4,DiffPointsPGLag4,isPlayoff
0,2008,1610612747,1.0,0.695,7.3,7.0,0.512,-0.1,7.0,0.549,2.5,11.0,0.415,-3.0,1
1,2008,1610612743,8.0,0.610,3.7,6.0,0.549,1.6,4.0,0.537,0.2,7.0,0.598,2.0,1
2,2008,1610612759,3.0,0.683,4.8,3.0,0.707,8.4,1.0,0.768,6.8,2.0,0.720,7.8,1
3,2008,1610612757,10.0,0.500,-1.0,12.0,0.390,-4.3,15.0,0.256,-9.5,13.0,0.329,-4.0,1
4,2008,1610612745,5.0,0.671,4.7,5.0,0.634,4.9,12.0,0.415,-1.6,5.0,0.622,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2021,1610612747,7.0,0.583,2.8,1.0,0.732,5.8,10.0,0.451,-1.7,11.0,0.427,-1.5,0
206,2021,1610612758,12.0,0.431,-3.7,12.0,0.431,-2.0,9.0,0.476,-1.1,12.0,0.329,-7.0,0
207,2021,1610612757,6.0,0.583,1.8,8.0,0.473,-1.1,3.0,0.646,4.2,3.0,0.598,2.6,0
208,2021,1610612760,14.0,0.306,-10.6,5.0,0.611,2.0,6.0,0.598,3.4,4.0,0.585,3.4,0


In [38]:
# df_east.to_csv('df_east.csv')
# df_west.to_csv('df_west.csv')
# df_merged.to_csv('df_merged.csv')